In [4]:
# Generelle moduler og funksjonsbeskrivelser brukt i forelesningen
from numpy import sin, cos, pi, exp
import numpy.fft as fft
import numpy as np
import matplotlib.pyplot as plt
from Kildekode._07_Frekvensanalyse import *

%matplotlib ipympl

<img src="NTNU_Logo.png" align="left" style="width: 30%">
<br clear="all" />
<br></br>

# Intro til Digitale Filtre

* **Emne IELEA2302 - Signalbehandling**
* **Uke 9/10, 2021**
* **Relevant Pensum:**
    * Kapittel 5 i læreboka DSP First
* **Underviser: Kai Erik Hoff**

# Tema:
* Signal og System
* Digitale Filtre - hva og hvorfor?
* Bruk av digitale filtre

#### Repetisjon
* Eksempeloppgaver


# Signal og System

### Generell terminologi

* **Signal:**
    * En funksjon som formidler informasjon om et fenomen.
* **System:**
    * Tar imot inngangssignal og produserer et nytt utgangssignal
    
    
* Eksempel på system:
<img src="Figurer/10_Digitale Filtre Intro/Fig0_DSPSystem.png" style="width: 80%; margin-left=100px" />

# LTI-system

* Vi skal i hovedsak begrense oss til å snakke om system som er ***lineære*** og ***tidsinvariante***.
    * Slike filtre omtales som ***LTI-system***
    * Det mest sentrale kjennetegnet på LTI-system er at sinusformede inngangssignal ***alltid*** vil gi sinusformede utgangssignal.
        * Utgangssignalet kan dog ha en annen *Amplitude* og *Fase*
    * *Mer om disse egenskapene i neste uke*

# Hva er et filter?
* I signalbehandlingssammenheng refererer et **filter** til et system som demper uønskede frekvenskomponenter i et signal.

    
<img src="Figurer/10_Digitale Filtre Intro/Fig2_FiltResp.png" style="width: 50%; margin-left=200px" />

# Analoge filtre
* Fysiske elektroniske kretser.
* Beskrives matematisk med en differensiallikning *(eller en transferfunksjon)*.
* Eksempel: RLC-krets:
<img src="Figurer/10_Digitale Filtre Intro/Fig1_AnalogFilt.png" style="width: 80%; margin-left=100px" />

# Digitale Filtre

* *Algoritme* for å regne ut hver sample av utgangssignalet $y[n]$
* Eksisterer kun som programvare
* Utgangssignalet er en ***lineær kombinasjon*** av signalverdier ved ulike tidspunkt
* Eksempel:
$$ y[n] = 0.25\cdot x[n] + 0.5\cdot x[n-1] + 0.25 \cdot x[n-2]$$
    * Utregning av sample nummer 5:
    $$y[5] = 0.25\cdot x[5] + 0.5\cdot x[4] + 0.25 \cdot x[3]$$
* Krever at programmet fører en "logg" over tidligere signalverdier

# Buksområder

* Bildebehandling
* Audioprosessering
* Kommunikasjonssystem
* Digitale kontrollere
* Mye mer

# Fordeler og ulemper

<br>
<div style="width: 100%;">
    <div style="float: left; width: 50%">
    <ul>
        <b> Fordeler </b>
        <br>
        <li> Tillater filterresponser som ikke kan oppnås med analoge filtre </li>
        <li> Kostnadseffektivt </li>
        <li> Enkle å modifisere </li>
    </ul>
    </div>
    <div style="float: right; width: 50%">
    <ul>
        <b> Ulemper</b>
        <br>
        <li> Kan tilføre ekstra forsinkelse i systemet </li>
        <li> Fungerer kun innenfor et frekvensbånd gitt av samplingsfrekvensen </li>
    </ul>
    </div>
</div> 

# Differanseligning

* Matematisk beskrivelse av et digitalt filter basert på ***filterkoeffisienter*** $a_k$ og $b_k$.
* Beskriver forholdet mellom inn- og utgangssignal.

### $$ \sum_{k=0}^{N}a_k\cdot  y[n-k] = \sum_{k=0}^{M} b_k \cdot x[n-k]$$

* Digital filterdesign sentrerer seg rundt å finne passende verdier for filterkoeffisientene $a_k$ og $b_k$. 
    * Disse verdiene anvendes så *direkte* i filteralgoritmen uten behov for å balansere kretskomponenter.

# Hvordan bruke digitale filter

1. Regn ut filterkoeffisientene $a_k$ og $b_k$.
    * Her er det vanlig å bruke dataverktøy som Python eller Matlab
    * Det finnes mange ulike fremgangsmåter avhengig av bruk
    
    
2. Implementer filteralgoritmen på en digital enhet
    * Dette kan for eksempel være i form av et program på en Arduino mikrokontroller.
    

## Eksempel: Regn ut filterparametre i Python

In [7]:
import scipy.signal as sig


# Kode for Digitalt Fjerdeordens Butterworth Lavpassfilter
b, a = sig.butter(N = 5,              # Filterorden
                  Wn = 0.35,          # Digital Knekkfrekvens (x pi)
                  btype = 'lowpass',  # Lavpassfilter
                  output = 'ba'       # Returner filterkoeffisenter
                  )

print("b:", b, "\na:", a)

w, H = sig.freqz(b, a)
plt.close(1); plt.figure(1)
plt.plot(w/pi, abs(H))

b: [0.01290369 0.06451846 0.12903692 0.12903692 0.06451846 0.01290369] 
a: [ 1.         -1.47967143  1.40369177 -0.68085641  0.1918841  -0.0221299 ]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Eksempel: Implementasjon på Arduino
<pre>
<font color="#95a5a6">&#47;* A program to read from analog input periodically, filter the input using a digital butterworth filter,</font>
<font color="#95a5a6"> &nbsp;&nbsp;&nbsp;and print the filtered signal samples to the serial monitor. *&#47;</font>
<font color="#5e6d03">#include</font> <font color="#005c5f">&#34;IIR_filter.h&#34;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Contains IIR filter class</font>
<font color="#5e6d03">#include</font> <font color="#005c5f">&#34;Timer.h&#34;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Contains Timer class</font>

<font color="#00979c">const</font> <font color="#00979c">int</font> <font color="#000000">SIG_INPUT</font> <font color="#434f54">=</font> <font color="#000000">A0</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Signal Input Pin</font>
<font color="#00979c">const</font> <font color="#00979c">int</font> <font color="#000000">SIG_OUTPUT</font> <font color="#434f54">=</font> <font color="#000000">3</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Signal Output Pin</font>
<font color="#00979c">const</font> <font color="#00979c">unsigned</font> <font color="#00979c">int</font> <font color="#000000">T_S</font> <font color="#434f54">=</font> <font color="#000000">10</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Sample period</font>

<font color="#000000">IIR_filter</font> <font color="#000000">LP_Filt</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Create a filter object</font>
<font color="#000000">Timer</font> <font color="#000000">sampleTimer</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Timer to manage sampling intervals</font>

<font color="#00979c">void</font> <font color="#5e6d03">setup</font><font color="#000000">(</font><font color="#000000">)</font> <font color="#000000">{</font>
 &nbsp;<font color="#d35400">pinMode</font><font color="#000000">(</font><font color="#000000">SIG_INPUT</font><font color="#434f54">,</font> <font color="#00979c">INPUT</font><font color="#000000">)</font><font color="#000000">;</font>
 &nbsp;<font color="#d35400">pinMode</font><font color="#000000">(</font><font color="#000000">SIG_OUTPUT</font><font color="#434f54">,</font> <font color="#00979c">OUTPUT</font><font color="#000000">)</font><font color="#000000">;</font>

 &nbsp;<font color="#95a5a6">&#47;* Initialize myFilter object with filter coefficients and filter order as specified below *&#47;</font>
 &nbsp;<font color="#00979c">float</font> <font color="#000000">b</font><font color="#000000">[</font><font color="#000000">5</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">{</font><font color="#000000">0.03048527</font><font color="#434f54">,</font> <font color="#000000">0.12194108</font><font color="#434f54">,</font> <font color="#000000">0.18291162</font><font color="#434f54">,</font> &nbsp;<font color="#000000">0.12194108</font><font color="#434f54">,</font> <font color="#000000">0.03048527</font><font color="#000000">}</font><font color="#000000">;</font>
 &nbsp;<font color="#00979c">float</font> <font color="#000000">a</font><font color="#000000">[</font><font color="#000000">5</font><font color="#000000">]</font> <font color="#434f54">=</font> <font color="#000000">{</font><font color="#000000">1.</font><font color="#434f54">,</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">-</font><font color="#000000">1.17512564</font><font color="#434f54">,</font> <font color="#000000">0.92564873</font><font color="#434f54">,</font> <font color="#434f54">-</font><font color="#000000">0.31042133</font><font color="#434f54">,</font> <font color="#000000">0.04766256</font><font color="#000000">}</font><font color="#000000">;</font>
 &nbsp;<font color="#000000">LP_Filt</font><font color="#434f54">.</font><font color="#000000">initialize</font><font color="#000000">(</font><font color="#000000">b</font><font color="#434f54">,</font> <font color="#000000">a</font><font color="#000000">)</font><font color="#000000">;</font>
<font color="#000000">}</font>

<font color="#00979c">void</font> <font color="#5e6d03">loop</font><font color="#000000">(</font><font color="#000000">)</font> <font color="#000000">{</font>
 &nbsp;<font color="#5e6d03">if</font> <font color="#000000">(</font><font color="#000000">sampleTimer</font><font color="#434f54">.</font><font color="#000000">hasExpired</font><font color="#000000">(</font><font color="#000000">)</font><font color="#000000">)</font>
 &nbsp;<font color="#000000">{</font>
 &nbsp;&nbsp;&nbsp;<font color="#000000">sampleTimer</font><font color="#434f54">.</font><font color="#000000">start</font><font color="#000000">(</font><font color="#000000">T_S</font><font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
 &nbsp;&nbsp;&nbsp;<font color="#00979c">float</font> <font color="#000000">input_sample</font> <font color="#434f54">=</font> <font color="#000000">(</font><font color="#00979c">float</font><font color="#000000">)</font><font color="#d35400">analogRead</font><font color="#000000">(</font><font color="#000000">SIG_INPUT</font><font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Get Input Sample</font>
 &nbsp;&nbsp;&nbsp;<font color="#00979c">float</font> <font color="#000000">output_sample</font> <font color="#434f54">=</font> <font color="#000000">LP_Filt</font><font color="#434f54">.</font><font color="#000000">filter_sample</font><font color="#000000">(</font><font color="#000000">input_sample</font><font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Filter latest input sample</font>
 &nbsp;&nbsp;&nbsp;<font color="#d35400">analogWrite</font><font color="#000000">(</font><font color="#000000">SIG_OUTPUT</font><font color="#434f54">,</font> <font color="#d35400">round</font><font color="#000000">(</font><font color="#000000">output_sample</font><font color="#434f54">&#47;</font><font color="#000000">4</font><font color="#000000">)</font><font color="#000000">)</font><font color="#000000">;</font> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<font color="#434f54">&#47;&#47; Set output level</font>
 &nbsp;<font color="#000000">}</font>
<font color="#000000">}</font>

</pre>

# Planen videre:

* FIR filtre 
* Frekvensrespons
* IIR filtre
* Transferfunksjoner
* Poler og Nullpunkt
* Implementasjon av filtre

# Spørsmål?